input: TCGA VCFs, annotate where they are from, need easy way to access the analyte


![alt text](barcode.png "Logo Title Text 1")

Extract the allelic read counts from TCGA_VCF.pickle which has all the pre-preprocessed VCFs from 4 different pipelines. Use entire TCGA barcode for identification. 


Barcode from the good old TCGA website: https://tcga-data.nci.nih.gov/docs/publications/tcga/?

In [25]:
import pandas as pd
from tqdm import  tqdm
tqdm.pandas()


### load TCGA VCFs

In [26]:
tcgaOrignalVCFDir='./../Data/TCGA_VCF.pickle'


#!mkdir ./../Data
tcga_vcf=pd.read_pickle(tcgaOrignalVCFDir)

vcf_col='CHROM POS ID REF ALT QUAL FILTER INFO FORMAT TUMOR NORMAL'.split(' ')

tcga_vcf.columns=vcf_col

### load in pipeline informations

In [27]:
lgg_rest_meta_df=pd.read_pickle('../lgg_rest_meta.pickle')

In [28]:
print ('Pipeline counts\n',lgg_rest_meta_df['var_pipeline'].value_counts())

Pipeline counts
 varscan_dr       530
mutect           530
muse             530
somaticsniper    529
Name: var_pipeline, dtype: int64


In [5]:
#lgg_rest_meta_df_sub['submitter_id'].str.split('_')
tmpS2=lgg_rest_meta_df['submitter_id'].str.split('_')

extractCodeF=lambda s:'-'.join(s.split('-'))
lgg_rest_meta_df['TUMOR']=tmpS2.str[0].apply(extractCodeF)
lgg_rest_meta_df['NORMAL']=tmpS2.str[1].apply(extractCodeF)

### extract VCF for a given pipelines

In [41]:
"""

"""
query_var_pipeline='varscan_dr'

outDir=tcgaOrignalVCFDir.replace('.pickle',
                          '.{}.pickle'.format(query_var_pipeline)
                         )
tcga_indexed_barcode_outDir=tcgaOrignalVCFDir.replace('.pickle',
                          '.{}.barcode_indexed.pickle'.format(query_var_pipeline)
                         )

lgg_rest_meta_df_sub=lgg_rest_meta_df[lgg_rest_meta_df['var_pipeline']==query_var_pipeline]

In [22]:
#DP=89;GPV=1E0;SOMATIC;SPV=5.0789E-2;SS=2;SSC=12	

In [23]:
#lgg_rest_meta_df_sub

In [24]:
tcga_vcf_dir_fmt='/nrnb/users/btsui/Data/tcga_orig_vcf/{vcf_uuid}/{vcf_uuid}.vcf.gz'

def myF_TUMOR(tmpS):
    tmpDf3=pd.Series(dict(zip( tmpS['FORMAT'].split(':'), tmpS['TUMOR'].split(':'))))    
    return tmpDf3

def myF_NORMAL(tmpS):
    tmpDf3=pd.Series(dict(zip( tmpS['FORMAT'].split(':'), tmpS['NORMAL'].split(':'))))    
    return tmpDf3



def myF_INFO(tmpS):
    #tmpS=one_vcf_df['INFO'].iloc[0]
    tmpS3=pd.Series(tmpS['INFO'].split(';'))
    tmpL=tmpS3.str.split('=')
    m_len=tmpL.apply(lambda L:len(L)>=2)
    tmpL2=tmpL[m_len]#=[list(map(lambda L:len(L)>=2,tmpL))]
    tmpS3=pd.Series(index=tmpL2.str[0].values,data=tmpL2.str[1].values)
    return tmpS3

myDfDict={}
for i,vcf_uuid in tqdm(list(enumerate(lgg_rest_meta_df_sub['file_id']))): 
    tcga_vcf_dir=tcga_vcf_dir_fmt.format(vcf_uuid=vcf_uuid)
    one_vcf_df=tcga_vcf.loc[tcga_vcf_dir]
    #tumor_df=one_vcf_df.apply(myF_TUMOR,axis=1)
    #normal_df=one_vcf_df.apply(myF_NORMAL,axis=1)
    myDfDict[vcf_uuid]={'one_vcf_df':one_vcf_df,
                        'tumor_df':one_vcf_df.apply(myF_TUMOR,axis=1),
                        'normal_df':one_vcf_df.apply(myF_NORMAL,axis=1),
                        'INFO':one_vcf_df.apply(myF_INFO,axis=1)}
    #one_vcf_df_L.append(one_vcf_df)
    #tumor_df.append(tumor_df_L)
    #normal_df.append()
    #print (i)
    #if i >10:
    #    break
    

  5%|▍         | 25/530 [04:00<1:20:50,  9.61s/it]

KeyError: 'the label [/nrnb/users/btsui/Data/tcga_orig_vcf/5e1000a0-d296-41c2-9aae-45805b7d6a6f/5e1000a0-d296-41c2-9aae-45805b7d6a6f.vcf.gz] is not in the [index]'

In [ ]:
"""tmpS=one_vcf_df['INFO'].iloc[2]
tmpS3=pd.Series(tmpS.split(';'))
tmpL=tmpS3.str.split('=')
m_len=tmpL.apply(lambda L:len(L)>=2)
tmpL2=tmpL[m_len]#=[list(map(lambda L:len(L)>=2,tmpL))]
tmpS3=pd.Series(index=tmpL2.str[0].values,data=tmpL2.str[1].values)
tmpS3"""

In [ ]:
#myF_INFO()

In [ ]:
uuidToBarcodeMappingDf=lgg_rest_meta_df_sub.set_index('file_id')

In [ ]:
myL=[]
for myUUID,tmpDict in myDfDict.items():
    #tmpDict=myDfDict[myUUID]
    
    tmpL=list(tmpDict.values())
    
    mergedDf=pd.concat(list([tmpL[0].reset_index(),tmpL[1],tmpL[2],tmpL[3]]),axis=1,
                       keys=list(tmpDict.keys()),
              names=['Group','VCF_columns'])
    
    myL.append(mergedDf)

In [ ]:
uuidToBarcodeMappingDf_algnedDf=uuidToBarcodeMappingDf.loc[(myDfDict.keys())]#['TUMOR']

In [ ]:
access_names=['TUMOR','NORMAL']
accessKeys=uuidToBarcodeMappingDf_algnedDf[access_names].apply(tuple,axis=1).tolist()

In [ ]:
mergedDf_all=pd.concat(myL,axis=0,keys=accessKeys,names=access_names
         )

In [ ]:
#CHROM: 208248388, POS: 208248388 
queryChrom='chr2'
queryCorrdinate=208248388
m_pos=mergedDf_all['one_vcf_df']['POS']==queryCorrdinate
m_chrom=mergedDf_all['one_vcf_df']['CHROM']==queryChrom

In [ ]:
mergedDf_all.to_pickle(outDir)

In [37]:
preprocessed_vcfs=pd.read_pickle(outDir)

In [38]:
preprocessed_vcfs_normal_df=preprocessed_vcfs[['normal_df','one_vcf_df']]#.set_index(level=0)
preprocessed_vcfs_normal_df.index=preprocessed_vcfs_normal_df.index.get_level_values('NORMAL')

preprocessed_vcfs_normal_df.columns=pd.MultiIndex.from_arrays([
                preprocessed_vcfs_normal_df.columns.get_level_values('Group').str.replace('normal_df|tumor_df','vcf_info'),
                          preprocessed_vcfs_normal_df.columns.get_level_values('VCF_columns')])


preprocessed_vcfs_normal_df.index.name='NAME'


In [39]:
preprocessed_vcfs_tumor_df=preprocessed_vcfs[['tumor_df','one_vcf_df']]#.set_index(level=0)
preprocessed_vcfs_tumor_df.index=preprocessed_vcfs_tumor_df.index.get_level_values('NORMAL')

preprocessed_vcfs_tumor_df.columns=pd.MultiIndex.from_arrays([
                preprocessed_vcfs_tumor_df.columns.get_level_values('Group').str.replace('normal_df|tumor_df','vcf_info'),
                          preprocessed_vcfs_tumor_df.columns.get_level_values('VCF_columns')])


preprocessed_vcfs_tumor_df.index.name='NAME'


In [43]:
tcga_indexed_barcode_df=pd.concat([preprocessed_vcfs_tumor_df],axis=0)

In [44]:
tcga_indexed_barcode_df.to_pickle(tcga_indexed_barcode_outDir)

In [46]:

tcga_indexed_barcode_df

Group                        vcf_info                                    \
VCF_columns                        GT GQ   DP   RD AD   FREQ        DP4   
NAME                                                                      
TCGA-DU-5870-10A-01D-1705-08      0/0  .   69   67  0     0%  38,29,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   67   66  0     0%  34,32,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   53   52  0     0%  19,33,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   45   43  2  4.44%  28,15,0,2   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   34   34  0     0%   3,31,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   32   32  0     0%   3,29,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   23   22  0     0%   6,16,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   40   40  0     0%  15,25,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .  104  104  0     0%  56,48,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .  127  122  5  3.94%  9,113,0,5   
TCGA-DU-5870-10A-01D-1705-08      0/0  .  103   97  2  2.02%  56,41,0,2   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   23   23  0     0%   1,22,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   59   59  0     0%  16,43,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   53   53  0     0%  12,41,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   28   28  0     0%   19,9,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   84   82  1   1.2%  42,40,0,1   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   46   46  0     0%   45,1,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   46   46  0     0%  12,34,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   27   27  0     0%   20,7,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   30   30  0     0%   4,26,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   29   28  1  3.45%   0,28,0,1   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   21   20  1  4.76%   20,0,1,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .  161  153  8  4.97%  54,99,0,8   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   17   17  0     0%   2,15,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   26   26  0     0%   26,0,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   27   26  1   3.7%   3,23,0,1   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   19   19  0     0%   19,0,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   29   29  0     0%  18,11,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   36   36  0     0%   7,29,0,0   
TCGA-DU-5870-10A-01D-1705-08      0/0  .   59   59  0     0%  46,13,0,0   
...                               ... ..  ...  ... ..    ...        ...   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   12   12  0     0%   10,2,0,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   12   12  0     0%   10,2,0,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   11   11  0     0%   10,1,0,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   51   50  1  1.96%   7,43,0,1   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   48   47  1  2.08%   4,43,0,1   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   28   28  0     0%  17,11,0,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   44   43  1  2.27%   38,5,0,1   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   60   59  1  1.67%   4,55,0,1   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   62   61  1  1.61%   4,57,0,1   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   28   27  0     0%   23,4,0,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   33   32  1  3.03%  11,21,1,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   33   32  1  3.03%  12,20,1,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   34   34  0     0%   33,1,0,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   47   47  0     0%  30,17,0,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   25   24  1     4%   9,15,0,1   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   12   12  0     0%    3,9,0,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   52   52  0     0%  30,22,0,0   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   55   54  1  1.82%  33,21,0,1   
TCGA-DU-7018-10A-01D-2024-08      0/0  .   56   54  2  3.57%  33,21,1,1   
TCGA-DU-7018-10A-01D-2024

In [45]:
tcga_indexed_barcode_outDir

'./../Data/TCGA_VCF.varscan_dr.barcode_indexed.pickle'

In [ ]:
asdasdasd

### Scratch

In [ ]:
mergedDf_all[m_pos&m_chrom]

In [ ]:
queryChr='2'
m_chr=mergedDf_all.index.get_level_values('CHROM')==queryChr
targetPos=208248388

In [ ]:
mergedDf_all.groupby(['TUMOR','NORMAL']).head()

In [ ]:
myL=[]
for myUUID in mergedDf.keys():
    myL.append([uuidToBarcodeMappingDf.loc[myUUID]['TUMOR']],
              )

In [ ]:
mergedDf2=pd.concat([mergedDf],axis=0,keys=[
        [uuidToBarcodeMappingDf.loc[myUUID]['TUMOR']],
    [uuidToBarcodeMappingDf.loc[myUUID]['NORMAL']],
    [myUUID]],
    names=['TUMOR','NORMAL','VCF_UUID',''])

In [ ]:
#mergedDf

In [ ]:
#vcf_uuid='bcc7d112-2d5d-4ff1-a863-70f11d5b0f2c'#file_id from UUID

In [ ]:
"""
1. load in each VCF file, and extract the variant counts for tumor and normal individually, 
merged TCGA VCF: tcga_vcf
put them in a flat multi index layouts
2. load in the VCF file 
"""


one_vcf_df=tcga_vcf.loc[tcga_vcf_dir]#.str.split('GT:GQ:DP:RD:AD:FREQ:DP4')

In [ ]:

def myF_TUMOR(tmpS):
    tmpDf3=pd.Series(dict(zip( tmpS['FORMAT'].split(':'), tmpS['TUMOR'].split(':'))))    
    return tmpDf3


tumor_df=one_vcf_df.progress_apply(myF_TUMOR,axis=1)

def myF_NORMAL(tmpS):
    tmpDf3=pd.Series(dict(zip( tmpS['FORMAT'].split(':'), tmpS['NORMAL'].split(':'))))    
    return tmpDf3

normal_df=one_vcf_df.progress_apply(myF_NORMAL,axis=1)

In [ ]:
normal_df

In [ ]:
one_vcf_df.head()

In [ ]:
#outDf.iloc[0].stacl

In [ ]:
outDf

In [ ]:
tmpS=one_vcf_df.iloc[0]


In [ ]:
tmpDf3=pd.Series(dict(zip( tmpS['FORMAT'].split(':'), tmpS['TUMOR'].split(':'))))

In [ ]:
tmpDf3

In [ ]:
lgg_rest_meta_df_sub['TUMOR']
tcga_vcf.head()
'/nrnb/users/btsui/Data/tcga_orig_vcf/bcc7d112-2d5d-4ff1-a863-70f11d5b0f2c/bcc7d112-2d5d-4ff1-a863-70f11d5b0f2c.vcf.gz'